In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/leaf-classification/train.csv.zip
/kaggle/input/leaf-classification/sample_submission.csv.zip
/kaggle/input/leaf-classification/images.zip
/kaggle/input/leaf-classification/test.csv.zip


# **Prepare Data**

In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opentelemetry-sdk to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 12.7 MB/s eta 0:00:00
  

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import timm
import cv2
import albumentations as A
from glob import glob
import os
from autogluon.multimodal import MultiModalPredictor
from tqdm.notebook import tqdm
import zipfile

/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [4]:
zip_f = ['/kaggle/input/leaf-classification/test.csv.zip','/kaggle/input/leaf-classification/train.csv.zip','/kaggle/input/leaf-classification/sample_submission.csv.zip','/kaggle/input/leaf-classification/images.zip']
zip_d = ['test','train','sample_submission','images']
for i in range(4):
    os.makedirs(zip_d[i],exist_ok=True)
    with zipfile.ZipFile(zip_f[i], 'r') as zip_ref:
        zip_ref.extractall(zip_d[i])

In [7]:
train_df = pd.read_csv('train/train.csv')
test_df = pd.read_csv('test/test.csv')
sub_df = pd.read_csv('sample_submission/sample_submission.csv')
data = "/home/drasogun/DraSoGun/AI/SuperAI_ss6/Competitions/Leaf-Classification/images"

In [9]:
train_df['path'] = train_df['id'].astype(str).apply(lambda x: "images/images/" + x + '.jpg')
test_df['path'] = test_df['id'].astype(str).apply(lambda x: "images/images/" + x + '.jpg')

# **Explore Data**

In [10]:
train_df.head()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64,path
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391,images/images/1.jpg
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461,images/images/2.jpg
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930,images/images/3.jpg
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852,images/images/5.jpg
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250,images/images/6.jpg


In [11]:
test_df.head()

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64,path
0,4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.0,0.005859,...,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711,images/images/4.jpg
1,7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.0,0.033203,...,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922,images/images/7.jpg
2,9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.0,0.011719,...,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000,images/images/9.jpg
3,12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.0,0.003906,...,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789,images/images/12.jpg
4,13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.0,0.005859,...,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812,images/images/13.jpg


In [12]:
train_df.shape,test_df.shape

((990, 195), (594, 194))

In [13]:
train_df['species'].value_counts()

species
Acer_Opalus               10
Pterocarya_Stenoptera     10
Quercus_Hartwissiana      10
Tilia_Tomentosa           10
Quercus_Variabilis        10
                          ..
Ulmus_Bergmanniana        10
Quercus_Nigra             10
Salix_Intergra            10
Quercus_Infectoria_sub    10
Sorbus_Aria               10
Name: count, Length: 99, dtype: int64

# **Augmentation**

In [14]:
os.makedirs("train_aug", exist_ok=True)
aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=30, p=0.7),
    A.RandomBrightnessContrast(p=0.2),
])

In [15]:
new_rows = []
for idx,row in tqdm(train_df.iterrows(),total=len(train_df)):
    img_path = row['path']
    image = cv2.imread(img_path)
    for i in range(2):
        augment = aug(image=image)['image']
        new_filename = f"train_aug/{row['id']}_aug_{i}.jpg"
        cv2.imwrite(new_filename,augment)
        new_row = row.copy()
        new_row['path'] = new_filename
        new_row['id'] = f"{row['id']}_aug_{i}"
        new_rows.append(new_row)

  0%|          | 0/990 [00:00<?, ?it/s]

In [16]:
aug_df = pd.DataFrame(new_rows)
train_aug_df = pd.concat([train_df,aug_df],axis = 0).reset_index(drop=True)
print(f"Original shape: {train_df.shape}")
print(f"Augmented shape: {train_aug_df.shape}")

Original shape: (990, 195)
Augmented shape: (2970, 195)


In [17]:
train_aug_df.tail()

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64,path
2965,1581_aug_1,Alnus_Maximowiczii,0.001953,0.003906,0.000000,0.021484,0.078125,0.003906,0.007812,0.0,...,0.000977,0.004883,0.027344,0.016602,0.007812,0.000000,0.027344,0.000000,0.001953,train_aug/1581_aug_1.jpg
2966,1582_aug_0,Quercus_Rubra,0.000000,0.000000,0.046875,0.056641,0.009766,0.000000,0.000000,0.0,...,0.030273,0.000977,0.002930,0.014648,0.000000,0.041992,0.000000,0.001953,0.002930,train_aug/1582_aug_0.jpg
2967,1582_aug_1,Quercus_Rubra,0.000000,0.000000,0.046875,0.056641,0.009766,0.000000,0.000000,0.0,...,0.030273,0.000977,0.002930,0.014648,0.000000,0.041992,0.000000,0.001953,0.002930,train_aug/1582_aug_1.jpg
2968,1584_aug_0,Quercus_Afares,0.023438,0.019531,0.031250,0.015625,0.005859,0.019531,0.035156,0.0,...,0.000000,0.002930,0.000000,0.012695,0.000000,0.000000,0.023438,0.025391,0.022461,train_aug/1584_aug_0.jpg
2969,1584_aug_1,Quercus_Afares,0.023438,0.019531,0.031250,0.015625,0.005859,0.019531,0.035156,0.0,...,0.000000,0.002930,0.000000,0.012695,0.000000,0.000000,0.023438,0.025391,0.022461,train_aug/1584_aug_1.jpg


# **Model**

In [18]:
label = "species"
metric = "log_loss"

predictor = MultiModalPredictor(label = label,eval_metric= metric)

predictor.fit(
    train_data = train_aug_df,
    time_limit = 3600,
    presets = 'best_quality'
)

No path specified. Models will be saved in: "AutogluonModels/ag-20260122_030827"
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.12.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Sat Sep 27 10:16:09 UTC 2025
CPU Count:          4
Pytorch Version:    2.8.0+cu126
CUDA Version:       12.6
GPU Memory:         GPU 0: 15.89/15.89 GB
Total GPU Memory:   Free: 15.89 GB, Allocated: 0.00 GB, Total: 15.89 GB
GPU Count:          1
Memory Avail:       29.21 GB / 31.35 GB (93.2%)
Disk Space Avail:   19.41 GB / 19.52 GB (99.5%)
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	First 10 (of 99) unique label values:  ['Acer_Opalus', 'Pterocarya_Stenoptera', 'Quercus_Hartwissiana', 'Tilia_Tomentosa', 'Quercus_Variabilis', 'Magnolia_Salicifolia', 'Quercus_Canariensis', 'Quercus_Rubra', 'Quercus_Brantii', 'Salix_Fragilis']
	If 'multiclass' is not the correct problem_t

model.safetensors:   0%|          | 0.00/788M [00:00<?, ?B/s]

GPU Count: 1
GPU Count to be Used: 1

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.py:231: Precision 16-mixed is not supported by the model summary.  Estimated model size in MB will not be accurate. Using 32 bits instead.

  | Name              | Type                | Params | Mode 
------------------------------------------------------------------
0 | model             | MultimodalFusionMLP | 199 M  | train
1 | validation_metric | MeanMetric          | 0      | train
2 | loss_func         | CrossEntropyLoss    | 0      | train
------------------------------------------------------------------
199 M     Trainable params
0         Non-trainable params
199 M     Total params
796.668   Total estimated model params size (MB)
547       Modules in train mode
0         Modules in

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

AutoMM has created your model. 🎉🎉🎉

To load the model, use the code below:
    ```python
    from autogluon.multimodal import MultiModalPredictor
    predictor = MultiModalPredictor.load("/kaggle/working/AutogluonModels/ag-20260122_030827")
    ```

If you are not satisfied with the model, try to increase the training time, 
adjust the hyperparameters (https://auto.gluon.ai/stable/tutorials/multimodal/advanced_topics/customization.html),
or post issues on GitHub (https://github.com/autogluon/autogluon/issues).




# **Evaluate**

In [20]:
predictor.evaluate(train_aug_df)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

{'log_loss': 0.007884235365867371}

# **Submission**

In [21]:
pred = predictor.predict_proba(test_df)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [22]:
sub_df.head()

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,...,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101
1,7,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,...,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101
2,9,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,...,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101
3,12,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,...,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101
4,13,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,...,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101,0.010101


In [24]:
test_df

,id,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64,path
0,4,0.019531,0.009766,0.078125,0.011719,0.003906,0.015625,0.005859,0.000000,0.005859,...,0.000000,0.015625,0.000977,0.015625,0.0,0.0,0.000000,0.003906,0.053711,images/images/4.jpg
1,7,0.007812,0.005859,0.064453,0.009766,0.003906,0.013672,0.007812,0.000000,0.033203,...,0.000000,0.006836,0.001953,0.013672,0.0,0.0,0.000977,0.037109,0.044922,images/images/7.jpg
2,9,0.000000,0.000000,0.001953,0.021484,0.041016,0.000000,0.023438,0.000000,0.011719,...,0.000000,0.000977,0.000000,0.000000,0.0,0.0,0.015625,0.000000,0.000000,images/images/9.jpg
3,12,0.000000,0.000000,0.009766,0.011719,0.017578,0.000000,0.003906,0.000000,0.003906,...,0.015625,0.002930,0.036133,0.013672,0.0,0.0,0.089844,0.000000,0.008789,images/images/12.jpg
4,13,0.001953,0.000000,0.015625,0.009766,0.039062,0.000000,0.009766,0.000000,0.005859,...,0.042969,0.016602,0.010742,0.041016,0.0,0.0,0.007812,0.009766,0.007812,images/images/13.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
589,1576,0.000000,0.000000,0.003906,0.015625,0.041016,0.000000,0.017578,0.000000,0.005859,...,0.000000,0.004883,0.000000,0.003906,0.0,0.0,0.018555,0.000000,0.000977,images/images/1576.jpg
590,1577,0.000000,0.003906,0.003906,0.005859,0.017578,0.000000,0.017578,0.005859,0.000000,...,0.004883,0.004883,0.002930,0.009766,0.0,0.0,0.090820,0.000000,0.016602,images/images/1577.jpg
591,1579,0.017578,0.029297,0.015625,0.013672,0.003906,0.015625,0.025391,0.000000,0.000000,...,0.000000,0.028320,0.000000,0.001953,0.0,0.0,0.000000,0.042969,0.006836,images/images/1579.jpg
592,1580,0.013672,0.009766,0.060547,0.025391,0.035156,0.025391,0.039062,0.000000,0.003906,...,0.000000,0.000977,0.000000,0.011719,0.0,0.0,0.000000,0.011719,0.018555,images/images/1580.jpg


In [25]:
sub = pd.DataFrame()
sub['id'] = test_df['id']
sub = pd.concat([sub, pred], axis=1)
sub = sub[sub_df.columns]

In [26]:
sub.head()

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,...,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
0,4,2.180242e-11,8.864711e-09,2.605483e-08,2.579778e-09,7.099463e-05,3.261631e-08,6.147323e-08,1.793954e-09,6.626407e-09,...,1.436993e-07,7.235167e-09,1.280153e-06,3.162043e-07,3.941970e-08,9.165356e-08,9.199854e-09,6.147323e-08,9.514706e-06,1.531165e-07
1,7,4.141120e-07,2.364505e-06,1.337876e-07,1.004622e-06,2.131464e-06,9.786650e-09,1.089812e-07,6.927267e-08,1.064567e-07,...,3.591383e-06,1.711843e-06,1.247038e-07,6.476834e-07,1.167265e-05,2.529700e-05,1.016464e-06,3.774216e-07,3.770532e-07,4.365359e-07
2,9,6.070095e-08,9.998810e-01,6.373843e-08,2.477717e-05,1.243258e-06,1.745966e-07,2.001705e-08,5.485511e-06,7.894915e-07,...,3.526852e-08,9.698537e-09,1.589235e-07,2.348020e-07,2.370483e-07,2.605232e-09,5.700624e-10,7.869470e-09,2.008858e-07,1.105777e-06
3,12,1.872616e-08,1.646872e-05,8.506740e-09,4.536255e-08,2.272758e-06,1.499494e-05,5.537094e-07,6.602183e-06,3.717429e-07,...,2.091719e-06,5.012128e-07,4.096776e-07,1.574294e-06,5.146809e-06,1.040389e-07,1.105695e-06,6.021310e-08,2.397810e-07,3.610639e-06
4,13,1.413940e-07,8.169724e-06,8.458262e-09,7.883967e-09,1.951301e-08,4.576961e-08,7.710010e-08,1.353314e-07,4.690778e-07,...,1.018270e-08,2.549951e-08,2.907322e-06,1.163596e-04,3.624752e-07,2.674937e-08,3.056822e-07,1.232708e-10,4.291267e-08,5.999597e-07


In [28]:
sub.to_csv('submission.csv', index=False)